In [1]:
import random
import numpy as np
import copy

In [2]:
graph = np.array([[0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [3, 0, 0, 7, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 7, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0],
                   [0, 0, 9, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [2, 0, 0, 0, 4, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0],
                   [0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 6, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 8, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 9, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 1],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [3]:
def make_pheromone_graph(graph):
    pheromone_graph = np.array([])
    for i in range(graph.shape[0]):
        neighbours = {}
        for j in range(graph.shape[1]):
            if graph[i][j] == 0:
                continue
            neighbours[j] = {
                "from": i,
                "to": j,
                "weight": graph[i][j],
                "pheromones": 0.1
            }
        pheromone_graph = np.append(pheromone_graph, neighbours)
    return pheromone_graph

In [4]:
pheromone_graph = make_pheromone_graph(graph)
print(pheromone_graph)

[{1: {'from': 0, 'to': 1, 'weight': 3, 'pheromones': 0.1}, 5: {'from': 0, 'to': 5, 'weight': 2, 'pheromones': 0.1}}
 {0: {'from': 1, 'to': 0, 'weight': 3, 'pheromones': 0.1}, 3: {'from': 1, 'to': 3, 'weight': 7, 'pheromones': 0.1}, 7: {'from': 1, 'to': 7, 'weight': 1, 'pheromones': 0.1}}
 {4: {'from': 2, 'to': 4, 'weight': 9, 'pheromones': 0.1}}
 {1: {'from': 3, 'to': 1, 'weight': 7, 'pheromones': 0.1}, 9: {'from': 3, 'to': 9, 'weight': 5, 'pheromones': 0.1}}
 {2: {'from': 4, 'to': 2, 'weight': 9, 'pheromones': 0.1}, 5: {'from': 4, 'to': 5, 'weight': 4, 'pheromones': 0.1}}
 {0: {'from': 5, 'to': 0, 'weight': 2, 'pheromones': 0.1}, 4: {'from': 5, 'to': 4, 'weight': 4, 'pheromones': 0.1}, 6: {'from': 5, 'to': 6, 'weight': 6, 'pheromones': 0.1}}
 {5: {'from': 6, 'to': 5, 'weight': 6, 'pheromones': 0.1}}
 {1: {'from': 7, 'to': 1, 'weight': 1, 'pheromones': 0.1}, 8: {'from': 7, 'to': 8, 'weight': 2, 'pheromones': 0.1}}
 {7: {'from': 8, 'to': 7, 'weight': 2, 'pheromones': 0.1}, 9: {'from': 8

In [5]:
def add_neighbours(visited, neighbours, pheromone_graph):
    for node in visited:
        for branch in pheromone_graph[node]:
            if pheromone_graph[node][branch]['to'] not in visited:
                neighbours.append(pheromone_graph[node][branch])

In [6]:
def traverse(i, pheromone_graph, alpha, beta, k):
    n = pheromone_graph.shape[0]
    tree = [i]
    tree_branches = []
    tree_weight = 0
    visited = [i]
    while len(visited) < k:
        neighbours = []
        add_neighbours(visited, neighbours, pheromone_graph)
        values = [v['pheromones']**alpha / v['weight']**beta for v in neighbours]
        chosen_neighbour = random.choices(neighbours, weights = values, k = 1)[0]
        tree.append(chosen_neighbour['to'])
        tree_branches.append(chosen_neighbour)
        visited.append(chosen_neighbour['to'])
        tree_weight += chosen_neighbour['weight']
    
    if tree_weight != float('inf'):
        return tree, tree_branches, tree_weight
    else:
        print("There is no such tree in this graph")

In [7]:
traverse(0, pheromone_graph, 0.1, 0.1, 3)

([0, 5, 1],
 [{'from': 0, 'to': 5, 'weight': 2, 'pheromones': 0.1},
  {'from': 0, 'to': 1, 'weight': 3, 'pheromones': 0.1}],
 5)

In [8]:
def aco(pheromone_graph, num_ants, rho, num_iters, theta, alpha, beta, k):
    n = pheromone_graph.shape[0]
    best_tree = None

    for i in range(pheromone_graph.shape[0]):
        pheromone_graph = make_pheromone_graph(graph)
        node_best_tree = None
        
        for j in range(num_iters):
            trees = [traverse(i, pheromone_graph, alpha, beta, k) for _ in range(num_ants)]

            for u in range(n):
                for v in range(u + 1, n):
                    if v not in pheromone_graph[u]:
                        continue
                    pheromone_graph[u][v]['pheromones'] *= rho

            for tree, tree_branches, tree_weight in trees:
                delta = theta / tree_weight
                for branch in tree_branches:
                    branch['pheromones'] += delta

            ants_best_tree = min(trees, key = lambda c: c[2])
            if node_best_tree is None or ants_best_tree[2] < node_best_tree[2]:
                node_best_tree = copy.deepcopy(ants_best_tree)

        if best_tree is None or node_best_tree[2] < best_tree[2]:
                best_tree = copy.deepcopy(node_best_tree)

    print(best_tree)

In [9]:
aco(pheromone_graph, 10, 0.9, 100, 0.9, 1.5, 100.0, 10)

([0, 5, 1, 7, 8, 9, 10, 4, 3, 11], [{'from': 0, 'to': 5, 'weight': 2, 'pheromones': 0.4114285714285714}, {'from': 0, 'to': 1, 'weight': 3, 'pheromones': 0.4114285714285714}, {'from': 1, 'to': 7, 'weight': 1, 'pheromones': 0.4114285714285714}, {'from': 7, 'to': 8, 'weight': 2, 'pheromones': 0.4114285714285714}, {'from': 8, 'to': 9, 'weight': 1, 'pheromones': 0.4114285714285714}, {'from': 9, 'to': 10, 'weight': 4, 'pheromones': 0.4114285714285714}, {'from': 5, 'to': 4, 'weight': 4, 'pheromones': 0.42142857142857143}, {'from': 9, 'to': 3, 'weight': 5, 'pheromones': 0.42142857142857143}, {'from': 10, 'to': 11, 'weight': 6, 'pheromones': 0.2507142857142857}], 28)
